In [1]:
# Sentyment analysis
# Autor: Erick Tornero
# Topic: Sentiment analysis, Word Embedding, RNN

In [2]:
import pandas as pd
import pyprind

In [3]:
# We will use the data cleaned of previous Work 
# For more details see:
# https://github.com/erickTornero/Intelligent-Systems/tree/master/w3
newdf = pd.read_csv('textcleaned.csv', index_col = 0)

In [46]:
newdf.head()

,review,sentiment
Id,,
0,teenager martha moxley maggie grace moves hig...,1
1,ok so really like kris kristofferson usual eas...,0
2,spoiler do not read this if you think about wa...,0
3,hi for all people who have seen this wonderful...,1
4,recently bought dvd forgetting just how much h...,0


In [4]:
# Get all the sentences, consider that a sentence is a complete review
sentences = []
for text in newdf['review']:
    sentences.append(text.split())

In [5]:
len(sentences)

50000

# Train own Word2vect

Train Word2Vect, that rely on the words of our dataset

In [6]:
SZ_EMB_WORD = 100

In [7]:
# workers: Threads, depends of the pc in this case 4.
# Size: Size of the vector: we'll test with 100
from gensim.models import word2vec
modelW2V = word2vec.Word2Vec(sentences, workers= 4,size=SZ_EMB_WORD,min_count=20,window=20)
modelW2V.init_sims(replace=True)
modelW2V.save('modelreviewfilms')

In [8]:
# Test some similar word2word
modelW2V.wv.most_similar('excellent')

/home/erick/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('outstanding', 0.848717212677002),
 ('exceptional', 0.8028290271759033),
 ('superb', 0.764314591884613),
 ('terrific', 0.755221962928772),
 ('fantastic', 0.7386986613273621),
 ('great', 0.6987577080726624),
 ('amazing', 0.6793755888938904),
 ('fine', 0.6771063804626465),
 ('brilliant', 0.6741026043891907),
 ('wonderful', 0.6676398515701294)]

In [9]:
# Each vector of each word correspond to a vector of 100x1
modelW2V.wv.get_vector('excellent').shape

(100,)

# Split into train & test data

In [10]:
X = newdf['review'].values
Y = newdf['sentiment'].values
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 20)

In [11]:
import numpy as np

In [12]:
count = 0
for xx in X_train:
    count += 1
print(count)

35000


In [15]:
X_train.shape[0]

35000

# Statistics of Words:

In [37]:
L = []
for review in X_train:
    L.append(review.count(' ') +1)

In [38]:
L = np.array(L)

In [39]:
L.mean()

173.58814285714286

In [40]:
L.max()

1844

In [41]:
L.min()

4

In [43]:
L.var()

16523.47514512245

In [44]:
# Standard Desviation.
L.std()

128.54367018691528

**Esto significa que la cantidad de palabras por review se encuentran entre: 45 - 300 palabras por review**
* Por tanto se utilizarán 300 palabras. Si un review tiene más palabras se truncara a 300 palabras, y si un review tiene menos que 300 palabras se rellenara con ceros.

Ya que generar toda la matriz de Word2Vec para todo el dataset ocuparía demasiada memoria, se generará la matriz por Batch, ya que el Word2Vec entrenado es de 100 elementos por palabra, el batch sería una matriz de tamaño **Batch_SZx300x100**

# Obtención del Batch:

Para facilitar la tarea en el entrenamiento, se creara un vector de secuencia con las 300 palabras que se guardara en disco.

In [53]:
modelW2V.wv.vocab.get('it').index

0

In [56]:
modelW2V.wv.vocab.get('characters').index

81

In [58]:
modelW2V.wv.index2word[81]

'characters'

**Definición de algunos parámetros**

In [61]:
batch_size = 100
sequence_size = 300
word_size = 100

In [90]:
def GenIndexArr(seq_sz, texto):
    texto = texto.split()
    a = np.zeros(1)
    ind = set(modelW2V.wv.index2word)
    for word in texto:
        if word in ind:
            a = np.append(a, modelW2V.wv.vocab.get(word).index)
    a = np.delete(a, 0)
    if a.shape[0] > seq_sz:
        a = a[:seq_sz]
    elif a.shape[0] < seq_sz:
        b = np.zeros(seq_sz - a.shape[0])
        a = np.append(a, b)
    a = a.reshape(1, a.shape[0])
    return a.astype(int)

In [91]:
texx = X_train[0]

In [92]:
ar = GenIndexArr(sequence_size, texx)

In [96]:
ar.shape

(1, 300)

In [88]:
ar[0,1]

999.0

In [97]:
Y_train

array([1, 0, 0, ..., 0, 1, 1])

In [73]:
modelW2V.wv.index2word[999]

'large'

In [104]:
# Get all Reviews in a single list:
# Get all the reviews in a single list

def GetSequenceMatrix(X, seq_sz):
    
    reviews = []
    for review in X:
        reviews.append(review)
    
    arr = np.zeros([X.shape[0], sequence_size])
    pbar = pyprind.ProgBar(len(reviews))
    for i in range(X.shape[0]):
        arr[i] = GenIndexArr(seq_sz, reviews[i])
        pbar.update()
    return arr


In [105]:
X_v_train = GetSequenceMatrix(X_train, sequence_size)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:56


In [106]:
X_v_test = GetSequenceMatrix(X_test, sequence_size)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


In [112]:
X_v_train.shape

(35000, 300)

In [113]:
X_v_test.shape

(15000, 300)

In [114]:
np.save('X_train_seq', X_v_train)
np.save('Y_train', Y_train)

In [115]:
np.save('X_test_seq', X_v_test)
np.save('Y_test', Y_test)

In [111]:
modelW2V.wv.index2word[int(X_v_train[0][0])]

'im'

In [148]:
from math import ceil

In [149]:
class Batches:
    from math import ceil
    def __init__(self, data_length, index = 0, batch_size = 100):
        self.index = index
        self.batch_size = batch_size
        self.data_length = data_length
        self.num_batches = ceil(data_length/batch_size)
    def GetNextBatch(self, X, Y):
        self.index += 1
        epochEnd = False
        if self.index == self.num_batches:
            self.index = 1
            epochEnd = True
        if self.index < self.num_batches:
            return (epochEnd,X[((self.index-1)*self.batch_size):(self.index*self.batch_size),:], Y[(self.index-1)*self.batch_size:self.index*self.batch_size])
        #elif self.index == self.num_batches:
        #    return (False, X[(self.index-1)*self.batch_size:,:], Y[(self.index-1)*self.batch_size:])
    def RestartBatches(self):
        self.index = 0;

# Create the model of RNN

In [117]:
import tensorflow as tf

labels = tf.placeholder(tf.float32, [batch_size, 2])

In [119]:
input_data = tf.placeholder(tf.int32, [batch_size, sequence_size])

In [132]:
data = tf.Variable(tf.zeros([batch_size, sequence_size, word_size]),dtype=tf.float32)
data = tf.nn.embedding_lookup(modelW2V.wv.vectors,input_data)

In [136]:
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(modelW2V.wv.vectors,GenIndexArr(sequence_size, X_train[0])).eval().shape)

(1, 300, 100)


In [133]:
modelW2V.wv.vectors.shape

(19559, 100)

In [137]:
lstm_units = 64

In [139]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [140]:
weight = tf.Variable(tf.truncated_normal([lstm_units, 2]))
bias = tf.Variable(tf.constant(0.1, shape=[2]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [141]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [142]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [143]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [161]:
y_tr_list = []
for x in Y_train:
    if x==0:
        y_tr_list.append([1, 0])
    elif x==1:
        y_tr_list.append([0,1])

In [146]:
len(y_tr_list)

35000

In [150]:
batch = Batches(data_length=len(y_tr_list))

In [173]:
# sess = tf.InteractiveSession()
# saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
    #    #Next Batch of reviews
        v, nextBatch, nextBatchLabels = batch.GetNextBatch(X_v_train, y_tr_list);
        sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})

    #    #Write summary to Tensorboard
        if (i % 50 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
    #        writer.add_summary(summary, i)
    for i in range(10):
        _, nextBatch, nextBatchLabels = batch_test.GetNextBatch(X_v_test, y_test_list);
        print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)
    
    sess.close()
#    #Save the network every 10,000 training iterations
#    if (i % 10000 == 0 and i != 0):
#        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
#        print("saved to %s" % save_path)
# writer.close()

Accuracy for this batch: 54.00000214576721
Accuracy for this batch: 43.99999976158142
Accuracy for this batch: 43.00000071525574
Accuracy for this batch: 46.00000083446503
Accuracy for this batch: 44.999998807907104
Accuracy for this batch: 44.999998807907104
Accuracy for this batch: 50.0
Accuracy for this batch: 50.999999046325684
Accuracy for this batch: 51.99999809265137
Accuracy for this batch: 50.0


In [158]:
accuracy

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [164]:
y_test_list = []
for x in Y_test:
    if x==0:
        y_test_list.append([1, 0])
    elif x==1:
        y_test_list.append([0,1])

In [165]:
len(y_test_list)

15000

In [166]:
batch_test = Batches(len(y_test_list))

In [172]:
iterations = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        _, nextBatch, nextBatchLabels = batch_test.GetNextBatch(X_v_test, y_test_list);
        print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)
    sess.close()

Accuracy for this batch: 56.99999928474426
Accuracy for this batch: 41.999998688697815
Accuracy for this batch: 50.0
Accuracy for this batch: 55.000001192092896
Accuracy for this batch: 46.99999988079071
Accuracy for this batch: 46.99999988079071
Accuracy for this batch: 43.99999976158142
Accuracy for this batch: 44.999998807907104
Accuracy for this batch: 47.999998927116394
Accuracy for this batch: 46.00000083446503
